In [9]:
##Autor: Ian Meza
#Fecha 11/08/2021
#Descripción: Secuencia para crear un archivo de estilos para GeoServer en formato .sld 


#Importación de librerías
import pandas as pd

#Funciones
def leerreferencia():
    a=[]
    with open('sld.txt', 'r') as f:
        a = f.read()
    f.closed
    return a
    
def generarrangos(minimo,maximo):
    diferencia = maximo - minimo
    div = diferencia/5
    
    r1i = minimo
    r1s = div+minimo
    
    r2i = r1s
    r2s = div+r2i
    
    r3i = r2s
    r3s = div+r3i
    
    r4i = r3s
    r4s = div+r4i
    
    r5i = r4s
    r5s = maximo
    
    r1 = str(r1i)+" - "+str(r1s)
    r2 = str(r2i)+" - "+str(r2s)
    r3 = str(r3i)+" - "+str(r3s)
    r4 = str(r4i)+" - "+str(r4s)
    r5 = str(r5i)+" - "+str(r5s)
    
    rangos = [r1,r2,r3,r4,r5]
    minimos = [r1i,r2i,r3i,r4i,r5i]
    maximos = [r1s,r2s,r3s,r4s,r5s]
    
    return rangos, minimos, maximos

def leerpaletas(paleta):
    paletas = pd.read_excel("Paletas SLD con Variables Censo.xlsx", sheet_name="Climáticas")
    paso = paletas["No"] == paleta
    paleta = paletas[paso]
    pal = paleta["Código"]
    return list(pal)

def sustitucion(minimo,maximo,variable, codpaleta):
    a = leerreferencia()
    inicio = a.split("<se:Rule>")[0]
    inicio = inicio.replace("Datos_Censo_2017_Comuna", "stats_temperatura_precipitacion_worldclim_comuna")
    final = a.split("<se:Rule>")[1].split("</se:Rule>")[1]
    referencia = "\n        <se:Rule>"+a.split("<se:Rule>")[1].split("</se:Rule>")[0]+"</se:Rule>"
    referencia = "        <se:Rule>"+a.split("<se:Rule>")[1].replace("COLECTIVAS", variable)
    try:
        codpaleta = diccionario[variable]
    except:
        pass
        
    rangos,minimos,maximos = generarrangos(minimo,maximo)
    
    paleta = leerpaletas(codpaleta)
    
    salida = []
    for i in range(5):
        
        ref = referencia.replace("<se:Name>4 - 55.4</se:Name>", "<se:Name>"+rangos[i]+"</se:Name>")
        ref = ref.replace("<se:Title>4 - 55.4</se:Title>", "<se:Title>"+rangos[i]+"</se:Title>")
        ref = ref.replace("<ogc:Literal>4</ogc:Literal>", "<ogc:Literal>"+str(minimos[i])+"</ogc:Literal>")
        ref = ref.replace("<ogc:Literal>55.4</ogc:Literal>", "<ogc:Literal>"+str(maximos[i])+"</ogc:Literal>")
        ref = ref.replace("fff5eb", str(paleta[i]))
        ref = ref.replace("</se:FeatureTypeStyle>", "")
        ref = ref.replace("</UserStyle>", "")
        ref = ref.replace("</NamedLayer>", "")
        ref = ref.replace("</StyledLayerDescriptor>", "")
        salida.append(ref)
    
    cuerpo = ""
    for i in salida:
        cuerpo = cuerpo+i
    salida2 = inicio+cuerpo+final
    return salida2

def guardararchivo(territorio, variable, minimo , maximo, codpaleta):

#   | p = provincia | r = región | c = comuna | m = manzana |
    text_file = open(f"temp_precipitacion_c{territorio}_{variable}.sld", "w")

    text_file.write(sustitucion(minimo,maximo,variable, codpaleta))

    text_file.close()

In [3]:
diccionario = {}
diccionario = {"v001":15, "v002":14, "v003":14, "v004":14, "v005":14, "v006":14, "v007":14, "v008":14, "v009":14, "v010":14, "v011":15,'v012':15,'v013':15,'v014':15,'v015':15,'v016':15,'v017':15,'v018':14, 'v019':14}

In [4]:
df = pd.read_excel("https://github.com/Sud-Austral/SLD/blob/main/stats_temperatura_precipitacion_worldclim_comuna.xlsx?raw=true")
df.columns[7:]
df = df.rename(columns={"Annual precipitation mm":"v001",
                        "Annual mean temperature °C":"v002",
                        "Isothermality %":"v003",
                        "Max temperature of warmest month °C":"v004",
                        "Mean diurnal range °C":"v005",
                        "Mean temperature of coldest quarter °C":"v006",
                        "Mean temperature of driest quarter °C":"v007",
                        "Mean temperature of warmest quarter °C":"v008",
                        "Mean temperature of wettest quarter °C":"v009",
                        "Min temperature of coldest month °C":"v010",
                        "Precipitation of coldest quarter mm":"v011",
                        'Precipitation of driest month mm':"v012",
                        'Precipitation of driest quarter mm':"v013",
                        'Precipitation of warmest quarter mm':"v014",
                        'Precipitation of wettest month mm':"v015",
                        'Precipitation of wettest quarter mm':"v016",
                        'Precipitation seasonality Coefficient of Variation':"v017",
                        'Temperature annual range °C':"v018",
                        'Temperature seasonality °C':"v019"})
df

,COMUNA,NOM_COMUNA,REGION,NOM_REGION,PROVINCIA,NOM_PROVIN,Descriptor,v001,v002,v003,...,v010,v011,v012,v013,v014,v015,v016,v017,v018,v019
0,10104,FRESIA,10,REGIÓN DE LOS LAGOS,101,LLANQUIHUE,Mean,1848.007819,9.687881,51.285379,...,3.770915,722.777170,65.116497,241.562158,251.440970,272.369038,739.893667,43.924941,13.659578,22.628905
1,10104,FRESIA,10,REGIÓN DE LOS LAGOS,101,LLANQUIHUE,3rd Qu.,1967.500000,10.400000,52.000000,...,4.400000,773.000000,67.000000,252.000000,271.000000,289.000000,796.500000,45.000000,14.200000,23.635000
2,10104,FRESIA,10,REGIÓN DE LOS LAGOS,101,LLANQUIHUE,Min.,1592.000000,6.400000,50.000000,...,0.600000,613.000000,57.000000,205.000000,205.000000,244.000000,648.000000,40.000000,12.300000,19.770000
3,10104,FRESIA,10,REGIÓN DE LOS LAGOS,101,LLANQUIHUE,1st Qu.,1746.000000,9.400000,51.000000,...,3.400000,674.000000,63.000000,233.000000,234.000000,259.000000,693.000000,43.000000,13.300000,21.770000
4,10104,FRESIA,10,REGIÓN DE LOS LAGOS,101,LLANQUIHUE,Max.,2124.000000,11.000000,54.000000,...,5.500000,859.000000,74.000000,265.000000,293.000000,303.000000,859.000000,47.000000,14.900000,25.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788,9211,VICTORIA,9,REGIÓN DE LA ARAUCANÍA,92,MALLECO,Mean,1573.716335,10.638247,54.469323,...,2.014980,718.623108,37.663745,135.715538,141.378486,268.019124,784.717928,65.409562,23.039363,36.203291
1789,9211,VICTORIA,9,REGIÓN DE LA ARAUCANÍA,92,MALLECO,3rd Qu.,1678.500000,11.100000,55.000000,...,2.400000,761.500000,44.000000,145.000000,155.500000,285.500000,841.000000,67.000000,23.200000,36.410000
1790,9211,VICTORIA,9,REGIÓN DE LA ARAUCANÍA,92,MALLECO,Max.,2408.000000,11.900000,56.000000,...,3.100000,1130.000000,51.000000,193.000000,193.000000,416.000000,1232.000000,70.000000,23.600000,37.350000
1791,9211,VICTORIA,9,REGIÓN DE LA ARAUCANÍA,92,MALLECO,1st Qu.,1398.500000,10.300000,54.000000,...,1.700000,630.000000,31.000000,123.000000,125.000000,235.000000,687.000000,64.000000,22.900000,35.970000


In [10]:
#Proceso final Precipitaciones
#df = pd.read_excel("https://github.com/Sud-Austral/SLD/blob/main/stats_temperatura_precipitacion_worldclim_comuna.xlsx?raw=true")
#df

#VARIABLES POSIBLES: PROVINCIA , REGION

for i in df["COMUNA"].unique():
    df_aux = df[df["COMUNA"] == i ]
    
    for j in df.columns[7:]:
        minimo = df_aux[j].min()
        maximo = df_aux[j].max()
        #codpaleta = pal
        #print(diccionario[j])
        guardararchivo(i,j,minimo,maximo,diccionario[j])

In [ ]:
#Proceso final
df = pd.read_excel("Datos_Censo_2017_Comuna.xlsx")

#VARIABLES POSIBLES: PROVINCIA , REGION

for i in df["PROVINCIA"].unique():
    df_aux = df[df["PROVINCIA"] == i ]
    
    for j in df.columns[6:-2]:
        minimo = df_aux[j].min()
        maximo = df_aux[j].max()
        #codpaleta = pal
        #print(diccionario[j])
        guardararchivo(i,j,minimo,maximo,diccionario[j])